In [ ]:
# Import libraries for sound and display
import pandas as pd
import numpy as np
from os.path import join as pjoin
import os
import matplotlib.pyplot as plt
import librosa
import librosa.display
import seaborn as sns

# Import libraries for ML
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM,BatchNormalization , GRU
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from keras.utils import to_categorical, plot_model
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, ReLU, LeakyReLU
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD

Load the data

In [ ]:
Cries = pd.read_csv("cries_halfsecond.csv")
print(Cries.shape)
Cries.head()



In [ ]:
print(Cries.Labels.value_counts())
#print(Cries_test.Labels.value_counts())
Cries["Labels"].unique()
#Cries["Labels"] = Cries["Labels"].map({"hungry": 0, "discomfort": 1, "burping": 2, "belly_pain": 3, "tired": 4}).astype(int) #mapping numbers
Cries.head()

Data preparation

In [ ]:
# Take all rows and cols without last col for X which include features
X = Cries.iloc[:, :-1].values
# Take last col for Y, which include the cry labels
Y = Cries["Labels"].values


Undersample training set

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'not minority')
x_resampled, y_resampled = rus.fit_resample(X, Y)

In [ ]:
print(x_resampled.shape, y_resampled.shape)
print(len(y_resampled))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=42, test_size=0.2, shuffle=True)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Visualize final training set

In [ ]:
from collections import Counter
from sklearn.preprocessing import LabelEncoder

yp = LabelEncoder().fit_transform(y_resampled)
# summarize distribution
counter = Counter(yp)
for k,v in counter.items():
 per = v / len(yp) * 100
 print('Class=%d, n=%d (%.3f%%)' % (k, v, per))
# plot the distribution
plt.bar(counter.keys(), counter.values())
plt.show()

First normalize data to make learning easier:

In [ ]:
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)


Create the encoder

In [ ]:
input_size = x_train.shape[1]
visible = Input(shape=(input_size,))
#first layer
e = Dense(input_size/2)(visible)
e = BatchNormalization()(e)
e = ReLU()(e)
#second layer
e = Dense(input_size/4)(e)
e = BatchNormalization()(e)
e = ReLU()(e)
#bottleneck
n_bottleneck = 100
bottleneck = Dense(n_bottleneck)(e)

Create decoder (easier to make changes when seperated)

In [ ]:
# define decoder, level 1
d = Dense(input_size/4)(bottleneck)
d = BatchNormalization()(d)
d = ReLU()(d)
# decoder level 2
d = Dense(input_size/2)(d)
d = BatchNormalization()(d)
d = ReLU()(d)
# output layer
output = Dense(input_size, activation='linear')(d)
# define autoencoder model
model = Model(inputs=visible, outputs=output)

In [ ]:
model.compile(optimizer='adam', loss="mse")

In [ ]:
plot_model(model, "autoencoder_architecture.png", show_shapes=True)

In [ ]:
history = model.fit(x_train, x_train, epochs=10, batch_size=16, verbose=2, validation_data=(x_test,x_test))

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
encoder = Model(inputs=visible, outputs=bottleneck)
plot_model(encoder, 'encoder_no_decode.png', show_shapes=True)
# save the encoder to file
encoder.save('encoder.keras')

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix


Decision tree classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_neighbors': [3, 5, 7, 9, 11, 15],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}
x_train, x_test, y_train, y_test = train_test_split(x_resampled, y_resampled, random_state=42, test_size=0.2, shuffle=True)
scaler = MinMaxScaler()
scaler.fit(x_train)

x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

#x_encoded = encoder.predict(x_train)
model = GridSearchCV(KNeighborsClassifier(), grid_params, cv=5, n_jobs=-1)
model.fit(x_train, y_train)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

#x_test_encoded = encoder.predict(x_test)
print(f'Model Score: {model.score(x_test, y_test)}')

y_predict = model.predict(x_test)

cm = confusion_matrix(y_test, y_predict)
plt.figure(figsize=(12, 10))

disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Belly-pain", "Burping", "Discomfort", "Hungry", "Tired"])
disp.plot()

encoder2 = OneHotEncoder()
y_en = encoder2.fit_transform(np.array(y_test).reshape(-1, 1)).toarray()

cm = pd.DataFrame(cm, index=[i for i in encoder2.categories_], columns=[i for i in encoder2.categories_])
plt.figure(figsize=(8, 6))
sns.heatmap(cm, linecolor="white", cmap="Blues", linewidths=1, annot=True, fmt=".2f")

plt.title("Confusion Matrix", size=20)
plt.xlabel("Predicted Labels", size=14)
plt.ylabel("Actual Labels", size=14)
plt.show()
print(classification_report(y_test, y_predict))
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_predict)}')

In [ ]:
# evaluate a model
def evaluate_model(X, y, model):
  # define evaluation procedure
  cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
  # evaluate model
  scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  return scores


x_train_encoded = encoder.predict(x_resampled)
x_test_encoded = encoder.predict(x_test)
#x_train_encoded = x_resampled
#x_test_encoded = x_test

#weights = {'belly_pain':2.0, 'burping':2.0, 'discomfort':1.0, 'hungry':1.0, 'tired':1.0}
rf = RandomForestClassifier(n_estimators=1000)
rf.fit(x_train_encoded, y_resampled)
y_pred = rf.predict(x_test_encoded)


#accuracy = accuracy_score(y_test, y_pred)
#accuracy = evaluate_model(x_train, y_train, rf)

#print(np.mean(accuracy), np.std(accuracy))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

cm = confusion_matrix(y_test, y_pred, normalize="all")
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Belly-pain", "Burping", "Discomfort", "Hungry", "Tired"])
disp.plot()
plt.show()
